# 1、数据的存储

举例1：从TXT文档中加载数据，向量化后存储到Chroma数据库

In [3]:
import os
import dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# 加载环境变量
dotenv.load_dotenv()

# 1、创建一个TextLoader的实例, 并将指定的文档加载
loader = TextLoader(
    file_path="./asset/load/09-ai1.txt",
    encoding="utf-8",
)

documents = loader.load()

# 2、创建文本拆分器并拆分文档
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

splitter_docs = text_splitter.split_documents(documents)

# print(splitter_docs)

# 3、获取嵌入模型
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# 4、将文档及嵌入模型传入到Chroma向量数据库中, 进行数据的存储
db = Chroma.from_documents(
    documents=splitter_docs,
    embedding=embedding_model,
)



1、当前的数据到底存到哪里去了？

如果我们使用的from_documents()中没有显式的指明存储位置的话, 则将当前的数据存储在内存中, 并缓存起来. 如果需要指明具体的存储位置, 需要设置参数persist_directory的值.

In [4]:
# 4、将文档及嵌入模型传入到Chroma向量数据库中, 进行数据的存储
db1 = Chroma.from_documents(
    documents=splitter_docs,
    embedding=embedding_model,
    persist_directory="./asset/chroma-1",
)

2、需要明确, 在向量数据库中, 不仅存储了数据（或文档）的向量, 而且还存储了数据（或文档）本身。

演示一下检索的需求：

In [5]:
# 5、查询：使用相似度查找
query = "人工智能的核心技术都有啥？"
docs = db.similarity_search(query)
print(docs[0].page_content)

3. 人工智能的核心技术
3.1 机器学习
机器学习是人工智能的核心技术之一，通过算法使计算机从数据中学习并做出决策。常见的机器学习算法包括监督学习、无监督学习和强化学习。监督学习通过标记数据进行训练，无监督学习则从未标记数据中寻找模式，强化学习则通过与环境交互来优化决策。
3.2 深度学习
深度学习是机器学习的一个子领域，通过多层神经网络进行特征提取和模式识别。深度学习在图像识别、自然语言处理、语音识别等领域取得了显著成果。常见的深度学习模型包括卷积神经网络（CNN）、循环神经网络（RNN）和长短期记忆网络（LSTM）。
3.3 自然语言处理
自然语言处理（NLP）是人工智能的一个重要分支，致力于使计算机能够理解和生成人类语言。NLP技术广泛应用于机器翻译、情感分析、文本分类等领域。近年来，基于深度学习的NLP模型（如BERT、GPT）在语言理解任务中取得了突破性进展。
3.4 计算机视觉
计算机视觉是人工智能的另一个重要分支，致力于使计算机能够理解和处理图像和视频。计算机视觉技术广泛应用于图像识别、目标检测、人脸识别等领域。深度学习模型（如CNN）在计算机视觉任务中取得了显著成果。

4. 人工智能的应用领域
4.1 医疗健康
人工智能在医疗健康领域的应用包括疾病诊断、药物研发、个性化医疗等。通过分析医学影像和患者数据，人工智能可以帮助医生更准确地诊断疾病，提高治疗效果。
4.2 金融
人工智能在金融领域的应用包括风险评估、欺诈检测、算法交易等。通过分析市场数据和交易记录，人工智能可以帮助金融机构做出更明智的决策，提高运营效率。
4.3 教育
人工智能在教育领域的应用包括个性化学习、智能辅导、自动评分等。通过分析学生的学习数据，人工智能可以为学生提供个性化的学习建议，提高学习效果。
4.4 交通
人工智能在交通领域的应用包括自动驾驶、交通管理、智能导航等。通过分析交通数据和路况信息，人工智能可以帮助优化交通流量，提高交通安全。


举例2：操作csv文档，并向量化

In [7]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os
import dotenv

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 获取嵌入模型
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 加载文档并拆分（第1次拆分）
loader = CSVLoader("./asset/load/03-load.csv", encoding='utf-8')
pages = loader.load_and_split()

#print(len(pages)) # 4
# 文本拆分（第2次拆分）
text_spliter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_spliter.split_documents(pages)

# 向量存储
db_path = './asset/chroma-2'
db = Chroma.from_documents(docs, embeddings, persist_directory=db_path)